In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [10]:
data = pd.read_csv('data/data_fusion_little.csv')
data = data.dropna()
data.head()

,Titre,Description,Auteurs,Image,Lien Google,Editeur,Date publication,infoLink,Genre,Nb scores,Id,Prix,User_id,Nom lecteur,revue/utilité,revue/score,revue/heure,revue/résumé,revue/texte
8,The Church of Christ: A Biblical Ecclesiology ...,In The Church of Christ: A Biblical Ecclesiolo...,['Everett Ferguson'],http://books.google.com/books/content?id=kVqRa...,http://books.google.nl/books?id=kVqRaiPlx88C&p...,Wm. B. Eerdmans Publishing,1996,http://books.google.nl/books?id=kVqRaiPlx88C&d...,['Religion'],5.0,0802841899,25.97,ARI272XF8TOL4,Christopher J. Bray,74/81,5.0,955411200,Ecclesiological Milestone,With the publication of Everett Ferguson's boo...
9,The Church of Christ: A Biblical Ecclesiology ...,In The Church of Christ: A Biblical Ecclesiolo...,['Everett Ferguson'],http://books.google.com/books/content?id=kVqRa...,http://books.google.nl/books?id=kVqRaiPlx88C&p...,Wm. B. Eerdmans Publishing,1996,http://books.google.nl/books?id=kVqRaiPlx88C&d...,['Religion'],5.0,0802841899,25.97,A36TPZSH8LBT1,haskell,2/3,5.0,1311465600,Early Christian development of the Church,Everett Ferguson approaches the subject of ear...
57,The Battleship Bismarck,The Bismarck is perhaps the most famous – and ...,['Stefan Draminski'],http://books.google.com/books/content?id=nxttD...,http://books.google.nl/books?id=nxttDwAAQBAJ&p...,Bloomsbury Publishing,20/09/2018,https://play.google.com/store/books/details?id...,['History'],1.0,0887402216,34.95,A30IUAABSEHEPI,Lawrence Duckles,9/9,3.0,1055980800,The Battleship Bismarck reviewed,This book is both a history and a photo album ...
58,The Battleship Bismarck,The Bismarck is perhaps the most famous – and ...,['Stefan Draminski'],http://books.google.com/books/content?id=nxttD...,http://books.google.nl/books?id=nxttDwAAQBAJ&p...,Bloomsbury Publishing,20/09/2018,https://play.google.com/store/books/details?id...,['History'],1.0,0887402216,34.95,A29SCEAL3JPMYW,Tim Grasshoff,0/0,4.0,1217808000,nice historical pictures,I like this book first one I've seen with so m...
74,Beginner's Yoruba (Hippocrene Beginner's Series),"""Beginner's Yoruba"" is now available with two ...",['Kayode J. Fakinlede'],http://books.google.com/books/content?id=xLe4n...,http://books.google.nl/books?id=xLe4nWzeSw0C&p...,Hippocrene Books,2005,http://books.google.nl/books?id=xLe4nWzeSw0C&d...,['Foreign Language Study'],1.0,0781810698,19.77,A1F0EV2MBF208I,"Olena Y. Rabinowitz ""Book lover""",14/14,4.0,1156291200,Very authentic,This is my first encounter with Yoruba and I h...


In [11]:
from sklearn.model_selection import train_test_split

X_polarite = data[['Titre', 'revue/texte']]
Y_polarite = data['revue/score']

In [19]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

def encode_simple(X_polarite, Y_polarite):
    # Séparer les données en train et test
    X_train_polarite, X_test_polarite, Y_train_polarite, Y_test_polarite = train_test_split(
        X_polarite, Y_polarite, test_size=0.2, random_state=42
    )

    # Copier explicitement pour éviter les warnings
    X_train_polarite = X_train_polarite.copy()
    X_test_polarite = X_test_polarite.copy()

    # Initialiser l'encodeur avec gestion des valeurs inconnues
    titre_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
    texte_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

    # Appliquer l'encodage
    X_train_polarite['Titre'] = titre_encoder.fit_transform(X_train_polarite[['Titre']]).astype(float)
    X_train_polarite['revue/texte'] = texte_encoder.fit_transform(X_train_polarite[['revue/texte']]).astype(float)

    X_test_polarite['Titre'] = titre_encoder.transform(X_test_polarite[['Titre']]).astype(float)
    X_test_polarite['revue/texte'] = texte_encoder.transform(X_test_polarite[['revue/texte']]).astype(float)

    return X_train_polarite, X_test_polarite, Y_train_polarite, Y_test_polarite


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from scipy.sparse import hstack

def encode_tfidf(X_polarite, Y_polarite):
    # Séparer les données en train et test
    X_train_polarite, X_test_polarite, Y_train_polarite, Y_test_polarite = train_test_split(
        X_polarite, Y_polarite, test_size=0.2, random_state=42
    )

    # Initialiser deux vectorizers TF-IDF (un pour chaque colonne)
    tfidf_titre = TfidfVectorizer()
    tfidf_texte = TfidfVectorizer()

    # Encodage des titres
    X_train_polarite_titre_tfidf = tfidf_titre.fit_transform(X_train_polarite['Titre'])
    X_test_polarite_titre_tfidf = tfidf_titre.transform(X_test_polarite['Titre'])

    # Encodage des textes
    X_train_polarite_texte_tfidf = tfidf_texte.fit_transform(X_train_polarite['revue/texte'])
    X_test_polarite_texte_tfidf = tfidf_texte.transform(X_test_polarite['revue/texte'])

    # Combiner les deux colonnes encodées
    X_train_polarite = hstack([X_train_polarite_titre_tfidf, X_train_polarite_texte_tfidf])
    X_test_polarite = hstack([X_test_polarite_titre_tfidf, X_test_polarite_texte_tfidf])

    return X_train_polarite, X_test_polarite, Y_train_polarite, Y_test_polarite


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import numpy as np
import os

# Encodage simple
X_train_polarite, X_test_polarite, Y_train_polarite, Y_test_polarite = encode_simple(X_polarite, Y_polarite)

# Convertir en numpy arrays pour TensorFlow
X_train_polarite = X_train_polarite.values.astype(np.float32)
X_test_polarite = X_test_polarite.values.astype(np.float32)
Y_train_polarite = Y_train_polarite.values.astype(np.float32)
Y_test_polarite = Y_test_polarite.values.astype(np.float32)

# Définir le modèle
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_polarite.shape[1],)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='linear')  # Utiliser 'linear' pour une régression
])

# Compiler le modèle
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Entraîner le modèle
history = model.fit(X_train_polarite, Y_train_polarite, epochs=50, batch_size=32, validation_split=0.2)

# Évaluer le modèle
loss, mae = model.evaluate(X_test_polarite, Y_test_polarite)
print(f"Test Loss: {loss}")
print(f"Test MAE: {mae}")

# Enregistrer le modèle (utiliser save() au lieu de pickle)
os.makedirs('models2', exist_ok=True)  # Créer le dossier si inexistant
model.save('models2/deepl_regression_simple.keras')



Epoch 1/50


C:\Users\Cedric\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


320/320 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 990462.0625 - mae: 615.6398 - val_loss: 76.8154 - val_mae: 7.7686
Epoch 2/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 14495.5215 - mae: 69.9925 - val_loss: 29.3426 - val_mae: 5.1269
Epoch 3/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4277.0596 - mae: 29.8479 - val_loss: 12.9890 - val_mae: 3.4203
Epoch 4/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1571.9329 - mae: 15.2611 - val_loss: 12.2349 - val_mae: 3.3126
Epoch 5/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1240.9801 - mae: 10.8610 - val_loss: 12.4768 - val_mae: 3.3492
Epoch 6/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 669.4883 - mae: 7.9034 - val_loss: 10.5625 - val_mae: 3.0518
Epoch 7/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 371.0244 - mae: 6.4013 - val_loss: 13.0780 - val_mae: 3.4391
Epoch 8/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 343.5607 - mae: 5.7319 - val_loss: 9.1667 - val_mae: 2.8230
Epoch 9/50
320/320 ━━━━━━━━━━━

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import numpy as np
import os

# Encodage TF-IDF
X_train_polarite, X_test_polarite, Y_train_polarite, Y_test_polarite = encode_tfidf(X_polarite, Y_polarite)

# Convertir la matrice sparse en numpy array
X_train_polarite = X_train_polarite.toarray().astype(np.float32)
X_test_polarite = X_test_polarite.toarray().astype(np.float32)

# Convertir les labels en numpy array
Y_train_polarite = np.array(Y_train_polarite).astype(np.float32)
Y_test_polarite = np.array(Y_test_polarite).astype(np.float32)

# Définir le modèle
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_polarite.shape[1],)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='linear')  # Utiliser 'linear' pour une régression
])

# Compiler le modèle
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Entraîner le modèle
history = model.fit(X_train_polarite, Y_train_polarite, epochs=50, batch_size=32, validation_split=0.2)

# Évaluer le modèle
loss, mae = model.evaluate(X_test_polarite, Y_test_polarite)
print(f"Test Loss: {loss}")
print(f"Test MAE: {mae}")

# Enregistrer le modèle (utiliser save() au lieu de pickle)
os.makedirs('models2', exist_ok=True)  # Créer le dossier si inexistant
model.save('models2/deepl_regression_tfidf.keras')


C:\Users\Cedric\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 17s 51ms/step - loss: 6.5686 - mae: 2.0605 - val_loss: 1.0902 - val_mae: 0.8611
Epoch 2/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 15s 48ms/step - loss: 1.3407 - mae: 0.9364 - val_loss: 0.9533 - val_mae: 0.7836
Epoch 3/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 15s 47ms/step - loss: 1.1274 - mae: 0.8577 - val_loss: 0.8993 - val_mae: 0.7537
Epoch 4/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 15s 47ms/step - loss: 0.9681 - mae: 0.7905 - val_loss: 0.9644 - val_mae: 0.7984
Epoch 5/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 15s 47ms/step - loss: 0.9135 - mae: 0.7662 - val_loss: 0.8701 - val_mae: 0.7282
Epoch 6/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 15s 48ms/step - loss: 0.8058 - mae: 0.7166 - val_loss: 0.8826 - val_mae: 0.7376
Epoch 7/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 15s 48ms/step - loss: 0.7542 - mae: 0.6938 - val_loss: 0.8890 - val_mae: 0.7487
Epoch 8/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 15s 47ms/step - loss: 0.6888 - mae: 0.6628 - val_loss: 0.8417 - val_mae: 0.6930
Epoch 9/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
import numpy as np
import os

# Encodage simple
X_train_polarite, X_test_polarite, Y_train_polarite, Y_test_polarite = encode_simple(X_polarite, Y_polarite)

# Vérifier les dimensions de X et Y avant le traitement
print(f"Dimensions de X_train_polarite : {X_train_polarite.shape}")
print(f"Dimensions de X_test_polarite : {X_test_polarite.shape}")
print(f"Dimensions de Y_train_polarite : {Y_train_polarite.shape}")
print(f"Dimensions de Y_test_polarite : {Y_test_polarite.shape}")

# Convertir en numpy arrays pour TensorFlow
X_train_polarite = X_train_polarite.astype(np.float32)
X_test_polarite = X_test_polarite.astype(np.float32)

# Vérifier les valeurs uniques dans Y_polarite
print(f"Valeurs uniques dans Y_polarite : {np.unique(Y_polarite)}")

# Recalage des labels pour qu'ils commencent à 0 (les classes sont [1, 2, 3, 4, 5], il faut les transformer en [0, 1, 2, 3, 4])
Y_train_polarite = Y_train_polarite - 1
Y_test_polarite = Y_test_polarite - 1

# Vérifier les valeurs uniques dans Y_polarite après recalage
print(f"Valeurs uniques dans Y_train_polarite après recalage : {np.unique(Y_train_polarite)}")

# S'assurer que les valeurs de Y_train_polarite et Y_test_polarite sont dans la plage [0, num_classes-1]
num_classes = len(np.unique(Y_polarite))  # Nombre total de classes
assert np.all(Y_train_polarite < num_classes), "Certains labels sont hors de portée dans Y_train_polarite."
assert np.all(Y_test_polarite < num_classes), "Certains labels sont hors de portée dans Y_test_polarite."

# One-hot encoding des labels multi-classes
Y_train_polarite = to_categorical(Y_train_polarite, num_classes)
Y_test_polarite = to_categorical(Y_test_polarite, num_classes)

# Vérifier la forme de Y après le one-hot encoding
print(f"Dimensions après One-Hot Encoding de Y_train_polarite : {Y_train_polarite.shape}")
print(f"Dimensions après One-Hot Encoding de Y_test_polarite : {Y_test_polarite.shape}")

# Définir le modèle pour classification multi-classes
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_polarite.shape[1],)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # Softmax pour multi-classes
])

# Compiler le modèle (perte adaptée aux multi-classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entraîner le modèle
history = model.fit(X_train_polarite, Y_train_polarite, epochs=50, batch_size=32, validation_split=0.2)

# Évaluer le modèle
loss, accuracy = model.evaluate(X_test_polarite, Y_test_polarite)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

# Enregistrer le modèle
os.makedirs('models2', exist_ok=True)  # Créer le dossier si inexistant
model.save('models2/deepl_classification_multi_simple.keras')


Dimensions de X_train_polarite : (12776, 2)
Dimensions de X_test_polarite : (3195, 2)
Dimensions de Y_train_polarite : (12776,)
Dimensions de Y_test_polarite : (3195,)
Valeurs uniques dans Y_polarite : [1. 2. 3. 4. 5.]
Valeurs uniques dans Y_train_polarite après recalage : [0. 1. 2. 3. 4.]
Dimensions après One-Hot Encoding de Y_train_polarite : (12776, 5)
Dimensions après One-Hot Encoding de Y_test_polarite : (3195, 5)
Epoch 1/50


C:\Users\Cedric\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


320/320 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4143 - loss: 629.7318 - val_accuracy: 0.6315 - val_loss: 1.4315
Epoch 2/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5764 - loss: 9.8628 - val_accuracy: 0.6315 - val_loss: 1.2715
Epoch 3/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6231 - loss: 3.0345 - val_accuracy: 0.6315 - val_loss: 1.1623
Epoch 4/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6264 - loss: 1.8026 - val_accuracy: 0.6315 - val_loss: 1.1153
Epoch 5/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6357 - loss: 1.3933 - val_accuracy: 0.6315 - val_loss: 1.0961
Epoch 6/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6317 - loss: 1.3469 - val_accuracy: 0.6315 - val_loss: 1.0881
Epoch 7/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6392 - loss: 1.2524 - val_accuracy: 0.6315 - val_loss: 1.0849
Epoch 8/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6294 - loss: 1.2898 - val_accuracy: 0.6315 - va

In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
import numpy as np
import os

# Encodage simple
X_train_polarite, X_test_polarite, Y_train_polarite, Y_test_polarite = encode_tfidf(X_polarite, Y_polarite)

# Vérifier les dimensions de X et Y avant le traitement
print(f"Dimensions de X_train_polarite : {X_train_polarite.shape}")
print(f"Dimensions de X_test_polarite : {X_test_polarite.shape}")
print(f"Dimensions de Y_train_polarite : {Y_train_polarite.shape}")
print(f"Dimensions de Y_test_polarite : {Y_test_polarite.shape}")

# Convertir en numpy arrays pour TensorFlow
X_train_polarite = X_train_polarite.astype(np.float32)
X_test_polarite = X_test_polarite.astype(np.float32)

# Vérifier les valeurs uniques dans Y_polarite
print(f"Valeurs uniques dans Y_polarite : {np.unique(Y_polarite)}")

# Recalage des labels pour qu'ils commencent à 0 (les classes sont [1, 2, 3, 4, 5], il faut les transformer en [0, 1, 2, 3, 4])
Y_train_polarite = Y_train_polarite - 1
Y_test_polarite = Y_test_polarite - 1

# Vérifier les valeurs uniques dans Y_polarite après recalage
print(f"Valeurs uniques dans Y_train_polarite après recalage : {np.unique(Y_train_polarite)}")

# S'assurer que les valeurs de Y_train_polarite et Y_test_polarite sont dans la plage [0, num_classes-1]
num_classes = len(np.unique(Y_polarite))  # Nombre total de classes
assert np.all(Y_train_polarite < num_classes), "Certains labels sont hors de portée dans Y_train_polarite."
assert np.all(Y_test_polarite < num_classes), "Certains labels sont hors de portée dans Y_test_polarite."

# One-hot encoding des labels multi-classes
Y_train_polarite = to_categorical(Y_train_polarite, num_classes)
Y_test_polarite = to_categorical(Y_test_polarite, num_classes)

# Vérifier la forme de Y après le one-hot encoding
print(f"Dimensions après One-Hot Encoding de Y_train_polarite : {Y_train_polarite.shape}")
print(f"Dimensions après One-Hot Encoding de Y_test_polarite : {Y_test_polarite.shape}")

# Définir le modèle pour classification multi-classes
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_polarite.shape[1],)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # Softmax pour multi-classes
])

# Compiler le modèle (perte adaptée aux multi-classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entraîner le modèle
history = model.fit(X_train_polarite, Y_train_polarite, epochs=50, batch_size=32, validation_split=0.2)

# Évaluer le modèle
loss, accuracy = model.evaluate(X_test_polarite, Y_test_polarite)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

# Enregistrer le modèle
os.makedirs('models2', exist_ok=True)  # Créer le dossier si inexistant
model.save('models2/deepl_classification_multi_tfidf.keras')


Dimensions de X_train_polarite : (12776, 78362)
Dimensions de X_test_polarite : (3195, 78362)
Dimensions de Y_train_polarite : (12776,)
Dimensions de Y_test_polarite : (3195,)
Valeurs uniques dans Y_polarite : [1. 2. 3. 4. 5.]
Valeurs uniques dans Y_train_polarite après recalage : [0. 1. 2. 3. 4.]
Dimensions après One-Hot Encoding de Y_train_polarite : (12776, 5)
Dimensions après One-Hot Encoding de Y_test_polarite : (3195, 5)
Epoch 1/50


C:\Users\Cedric\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


320/320 ━━━━━━━━━━━━━━━━━━━━ 17s 50ms/step - accuracy: 0.6167 - loss: 1.2120 - val_accuracy: 0.6315 - val_loss: 0.9146
Epoch 2/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 16s 50ms/step - accuracy: 0.6662 - loss: 0.7739 - val_accuracy: 0.6444 - val_loss: 0.8954
Epoch 3/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 19s 61ms/step - accuracy: 0.8081 - loss: 0.4830 - val_accuracy: 0.6318 - val_loss: 1.0517
Epoch 4/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 16s 50ms/step - accuracy: 0.8965 - loss: 0.2814 - val_accuracy: 0.6170 - val_loss: 1.2743
Epoch 5/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 16s 51ms/step - accuracy: 0.9383 - loss: 0.1747 - val_accuracy: 0.6252 - val_loss: 1.5050
Epoch 6/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 16s 49ms/step - accuracy: 0.9704 - loss: 0.1055 - val_accuracy: 0.5998 - val_loss: 1.7198
Epoch 7/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 16s 48ms/step - accuracy: 0.9815 - loss: 0.0641 - val_accuracy: 0.6252 - val_loss: 1.8702
Epoch 8/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 16s 48ms/step - accuracy: 0.9899 - loss: 0.0450 - val_accurac